In [3]:
import pandas as pd
import os
import tarfile
from PIL import Image
import torchvision 

#Extracting solar image data.
file = tarfile.open('solar_Flare.tar.gz') 
  
file.extractall('c:\\Users\\Shiyan Liu\\Github\\Solar_Flare_CNN') 
  
file.close() 

#Reading flare dataset.
flareDataset=pd.read_csv('flareDataset.csv')
print(flareDataset)

solar_Images=[f for f in os.listdir("img") if f.endswith(".jpg")]

      FlareNumber                T_REC  NOAA_AR  QUALITY  Longitude  Latitude  \
0               0  2010-05-01 23:48:00    11064        0         28        15   
1               0  2010-05-02 23:48:00    11064        0         15        16   
2               0  2010-05-03 23:48:00    11063        0         38        16   
3               0  2010-05-03 23:48:00    11064        0          2        15   
4               0  2010-05-03 23:48:00    11065        0         16        32   
...           ...                  ...      ...      ...        ...       ...   
8869            1  2017-09-07 15:00:00    12673        0         58         9   
8870            1  2017-09-07 23:24:00    12673        0         58         9   
8871            1  2017-09-08 04:12:00    12673        0         70         9   
8872            1  2017-09-08 10:48:00    12673        0         70         9   
8873            1  2017-09-08 22:00:00    12673        0         70         9   

       TOTUSJH        TOTBS

In [29]:
#Loading images
transformed_Images=[]
for imageFilePath in solar_Images:
    image=Image.open("img\\"+imageFilePath)
    tensor_Transform=torchvision.transforms.ToTensor()
    transformed_Image=tensor_Transform(image)
    transformed_Images.append(transformed_Image)

print(transformed_Images[0].shape)

torch.Size([3, 512, 512])
